# Анализ базы данных сервиса чтения книг

*Выпускной проект Яндекс Практикума. Часть III*

**Цель:** сформулировать ценностное предложение для создания нового мобильного приложения.

Данные — пять таблиц с данными: 
- о книгах,
- об авторах,
- об издательствах,
- о пользовательских оценках книг,
- о пользовательских обзорах на книги.

Заказчик: продакт-менеджер.

**Задачи:**
1. Посчитать количество книг, вышедших после 1 января 2000 года.
2. Для каждой книги посчитать количество обзоров и среднюю оценку.
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц.
4. Определить автора с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками).
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Оглавление
* Обзор данных
* Задача №1. Количество книг
* Задача №2. Обзоры и средняя оценка
* Задача №3. Продуктивное издательство
* Задача №4. Популярный автор
* Задача №5. Активность пользователей
* Итог

## Обзор данных

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

db_config = {'user': '***', 
             'pwd': '***', 
             'host': '***',
             'port': ***, 
             'db': '***'} 

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

Выведем первые строки датафреймов.

In [2]:
datasets = ['book', 'author', 'publisher', 'rating', 'review']

for item in datasets:
    query = f'''SELECT * 
                FROM {item}s 
                LIMIT 5
             '''
    display(pd.io.sql.read_sql(sql=text(query), con = con, index_col=f'{item}_id'))

,author_id,title,num_pages,publication_date,publisher_id
book_id,,,,,
1,546,'Salem's Lot,594,2005-11-01,93
2,465,1 000 Places to See Before You Die,992,2003-05-22,336
3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
5,125,1776,386,2006-07-04,268


,author
author_id,
1,A.S. Byatt
2,Aesop/Laura Harris/Laura Gibbs
3,Agatha Christie
4,Alan Brennert
5,Alan Moore/David Lloyd


,publisher
publisher_id,
1,Ace
2,Ace Book
3,Ace Books
4,Ace Hardcover
5,Addison Wesley Publishing Company


,book_id,username,rating
rating_id,,,
1,1,ryanfranco,4
2,1,grantpatricia,2
3,1,brandtandrea,5
4,2,lorichen,3
5,2,mariokeller,2


,book_id,username,text
review_id,,,
1,1,brandtandrea,Mention society tell send professor analysis. ...
2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
4,3,johnsonamanda,Finally month interesting blue could nature cu...
5,3,scotttamara,Nation purpose heavy give wait song will. List...


Создадим функцию для вывода запросов.

In [3]:
def get_query(data):
    display(pd.io.sql.read_sql(sql=text(data), con = con))

## Задача №1. Количество книг

Посчитаем количество книг, вышедших после 1 января 2000 года.

In [4]:
query_1 = f'''SELECT COUNT(books) 
              FROM books
              WHERE publication_date >= '2000-01-01'
         '''
get_query(query_1)

,count
0,821


Всего c 1 января 2000 года вышла 821 книга.

[назад в оглавление](#table)

## Задача №2. Обзоры и средняя оценка

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [5]:
query_2a = f'''
              SELECT title,
                     b.book_id,
                     COUNT(DISTINCT rev.review_id) AS review_count,
                     ROUND(AVG(rating), 2) AS avg_rating
              FROM books AS b
              LEFT JOIN ratings AS r ON r.book_id = b.book_id
              LEFT JOIN reviews AS rev ON rev.book_id = b.book_id
              GROUP BY b.book_id, title
              ORDER BY review_count DESC
         '''
get_query(query_2a)

,title,book_id,review_count,avg_rating
0,Twilight (Twilight #1),948,7,3.66
1,Water for Elephants,963,6,3.98
2,The Glass Castle,734,6,4.21
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.41
4,The Curious Incident of the Dog in the Night-Time,695,6,4.08
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.67
996,The Natural Way to Draw,808,0,3.00
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.00
998,Essential Tales and Poems,221,0,4.00


In [6]:
query_2b = f'''
              SELECT title,
                     b.book_id,
                     COUNT(DISTINCT rev.review_id) AS review_count,
                     ROUND(AVG(rating), 2) AS avg_rating
              FROM books AS b
              LEFT JOIN ratings AS r ON r.book_id = b.book_id
              LEFT JOIN reviews AS rev ON rev.book_id = b.book_id
              GROUP BY b.book_id, title
              ORDER BY avg_rating DESC
         '''
get_query(query_2b)

,title,book_id,review_count,avg_rating
0,Arrows of the Queen (Heralds of Valdemar #1),86,2,5.00
1,The Walking Dead Book One (The Walking Dead #...,901,2,5.00
2,Light in August,390,2,5.00
3,Wherever You Go There You Are: Mindfulness Me...,972,2,5.00
4,Captivating: Unveiling the Mystery of a Woman'...,136,2,5.00
...,...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,915,3,2.25
996,His Excellency: George Washington,316,2,2.00
997,Drowning Ruth,202,3,2.00
998,Junky,371,2,2.00


Количество обзоров на книгу — от 7 до 0. Средний рейтинг книг — от 5 до 1,5.

## Задача №3. Продуктивное издательство

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [7]:
query_3 = f'''
              SELECT p.publisher,
                     b.publisher_id,
                     COUNT(b.book_id) AS book_count
              FROM books AS b
              JOIN publishers AS p ON p.publisher_id = b.publisher_id
              WHERE num_pages > 50
              GROUP BY b.publisher_id, p.publisher
              ORDER BY book_count DESC
              LIMIT 1
         '''
get_query(query_3)

,publisher,publisher_id,book_count
0,Penguin Books,212,42


Издательство, выпустившее наибольшее число книг — Penguin Books: 42 книги.

## Задача №4. Популярный автор

Определить автора с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками).

In [8]:
query_4 = f'''
              SELECT author,
                     a.author_id,
                     AVG(rating)
              FROM authors AS a
              LEFT JOIN books AS b ON a.author_id = b.author_id
              LEFT JOIN ratings AS r ON r.book_id = b.book_id
              WHERE b.book_id IN (SELECT book_id
                                  FROM ratings
                                  GROUP BY book_id
                                  HAVING COUNT(rating_id) >= 50)
              GROUP BY author, a.author_id
              ORDER BY avg DESC
              LIMIT 1
         '''
get_query(query_4)

,author,author_id,avg
0,J.K. Rowling/Mary GrandPré,236,4.287097


Автор с самой высокой средней оценкой книг — Дж. К. Роулинг.

## Задача №5. Активность пользователей

Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [9]:
query_5 = f'''
              WITH count_review AS
              (SELECT rev.username,
                      COUNT(DISTINCT review_id)
               FROM reviews AS rev
               JOIN ratings AS rat ON rat.username = rev.username
               WHERE rev.username IN (SELECT username
                                      FROM ratings
                                      GROUP BY username
                                      HAVING COUNT(rating_id) > 48)
               GROUP BY rev.username)
              
              SELECT AVG(count)
              FROM count_review
               
         '''
get_query(query_5)

,avg
0,24.0


Среднее количество обзоров от пользователей, поставивших больше 48 оценок, — 24.

## Итог

Решены 5 задач по анализу базы данных сервиса чтения книг по подписке, состоящей из 5 таблиц. Результаты исследования следующие: 
1. База включает в себя 821 книгу, вышедшую с 1 января 2000 года.
2. Количество обзоров на каждую книгу составляет от 0 до 7. Рейтинг книг колеблется от 1,5 до 5 баллов. 
3. Наиболее продуктивное издательство в базе — Penguin Books, оно выпустило 42 книги (толщиной более 50 стр.).
4. Автор с самой высокой средней оценкой всех его книг — Дж. К. Роулинг (при подсчете учитывались только книги с 50 и более оценками).
5. В среднем пользователь, который ставит много оценок (больше 48) пишет 24 обзора.